### ドライブのマウント

In [13]:
# Googleドライブをマウント
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### ライブラリ・モジュールのインポート

In [14]:
# ライブラリの準備
!pip install timm
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from timm.scheduler import CosineLRScheduler
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import copy
import random
import pickle

### シード値の設定

In [15]:
# シード値を設定
def fix_seed(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

fix_seed(seed=1234)

### データセットの準備

In [16]:
# 平均値と標準偏差を計算するためのCIFAR100データセットの前処理を定義
pre_transform_cifar100 = transforms.Compose([
    transforms.ToTensor()
])

# 平均値と標準偏差を計算するためのCIFAR100データセットの読み込み
pre_train_dataset_cifar100 = datasets.CIFAR100(root='/content/data/', download=True, transform=pre_transform_cifar100)

# 平均値と標準偏差を計算するための変数を初期化
pre_mean_cifar100 = 0.0
pre_std_cifar100 = 0.0
pre_total_samples_cifar100 = len(pre_train_dataset_cifar100)

# データセットのすべてのデータポイントに対して平均値と標準偏差を計算
for data in pre_train_dataset_cifar100:
    pre_image, _ = data
    pre_mean_cifar100 += pre_image.mean(dim=(1, 2))  # テンソルのチャンネルごとに平均を計算
    pre_std_cifar100 += pre_image.std(dim=(1, 2))    # テンソルのチャンネルごとに標準偏差を計算

# データセット全体の平均値と標準偏差を計算
pre_mean_cifar100 /= pre_total_samples_cifar100
pre_std_cifar100 /= pre_total_samples_cifar100

print("データセット全体の平均値: ", pre_mean_cifar100)
print("データセット全体の標準偏差: ", pre_std_cifar100)

Files already downloaded and verified
データセット全体の平均値:  tensor([0.5071, 0.4866, 0.4409])
データセット全体の標準偏差:  tensor([0.2009, 0.1984, 0.2023])


In [17]:
# 学習用のCIFAR100データセットの前処理を定義
train_transform_cifar100 = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_cifar100, std=pre_std_cifar100)
    ])
# テスト用のCIFAR100データセットの前処理を定義
test_transform_cifar100 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=pre_mean_cifar100, std=pre_std_cifar100)
    ])

# 学習用のCIFAR100データセットの読み込み
train_dataset_cifar100 = torchvision.datasets.CIFAR100(root='/content/data/', train=True, transform=train_transform_cifar100, download=True)
# テスト用のCIFAR100データセットの読み込み
test_dataset_cifar100 = torchvision.datasets.CIFAR100(root='/content/data/', train=False, transform=test_transform_cifar100, download=True)

# 学習用のCIFAR100データローダーを作成
train_loader_cifar100 = torch.utils.data.DataLoader(dataset=train_dataset_cifar100, batch_size=512, shuffle=True, num_workers=2)
# テスト用のCIFAR100データローダーを作成
test_loader_cifar100 = torch.utils.data.DataLoader(dataset=test_dataset_cifar100, batch_size=512, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


## モデルの実装

In [18]:
# 1×1のサブネットワーク獲得用の畳み込みを定義
def supermaskconv1x1(in_channels, out_channels, stride=1):
    return SupermaskConv(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)

# 3×3のサブネットワーク獲得用の畳み込みを定義
def supermaskconv3x3(in_channels, out_channels, stride=1):
    return SupermaskConv(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)

In [19]:
# サブネットワーク獲得用のバッチ正則化として、非アフィン正規化を使用する（学習可能なパラメータを使用しない）
class NonAffineBatchNorm(nn.BatchNorm2d):
    def __init__(self, dim):
        super(NonAffineBatchNorm, self).__init__(dim, affine=False)

In [20]:
# 各重みにスコアを付与したスコアをソートしてtop k%を使用することでサブネットワークを獲得する
class GetSubnet(autograd.Function):
    @staticmethod
    def forward(ctx, scores, k):
        # スコアを複製する
        out = scores.clone()
        # スコアを昇順でソートする
        _, idx = scores.flatten().sort()
        # top k%以下のスコアの数
        j = int((1 - k) * scores.numel())
        # flat_outとoutは同じメモリを参照する（flat_outを変更するとoutにも影響する）
        flat_out = out.flatten()
        # top k%の要素を1にする
        flat_out[idx[j:]] = 1
        # top k%以外の要素を0にする
        flat_out[idx[:j]] = 0
        return out

    @staticmethod
    def backward(ctx, g):
        # 逆伝播時に勾配gをそのまま伝える
        return g, None

In [21]:
# サブネットワーク獲得用の畳み込みを定義
class SupermaskConv(nn.Conv2d):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # 重みと同じ形状のスコアを用意
        self.scores = nn.Parameter(torch.Tensor(self.weight.size()))
        # スコアを一様分布で初期化
        nn.init.kaiming_uniform_(self.scores, a=math.sqrt(5))
        # 重みの勾配を無効化する
        self.weight.requires_grad = False

    def _init_conv(self):
        # 重みを一様分布で初期化
        if self.init == 'kaiming_uniform':
            nn.init.kaiming_uniform_(self.weight, mode='fan_out', nonlinearity='relu')
        # 重みを正規分布で初期化
        elif self.init == 'kaiming_normal':
            nn.init.kaiming_normal_(self.weight, mode='fan_out', nonlinearity='relu')
        # 重みを符号つき定数で初期化
        elif self.init == 'signed_constant':
            fan = nn.init._calculate_correct_fan(self.weight, mode='fan_out')
            gain = nn.init.calculate_gain('relu')
            std = gain / math.sqrt(fan)
            self.weight.data = self.weight.data.sign() * std

    def set_init(self, init):
        # 初期化手法を設定
        self.init = init

    def set_prune_rate(self, prune_rate):
        # 刈り込み率を設定
        self.prune_rate = prune_rate

    @property
    def clamped_scores(self):
        # スコアとして非負の値を返すようにする（重要度を表す）
        return self.scores.abs()

    def forward(self, x):
        # サブネットワークを獲得
        subnet = GetSubnet.apply(self.clamped_scores, 1 - self.prune_rate)
        # サブネットワークでマスク
        w = self.weight * subnet
        x = F.conv2d(x, w, self.bias, self.stride, self.padding, self.dilation, self.groups)
        return x

In [22]:
# サブネットワーク獲得用のResidual Blocksを定義
class SupermaskBasicBlock(nn.Module):
    # 入力チャンネル数に対する出力チャンネル数の倍率
    expansion = 1

    def __init__(self, in_channels, channels, stride=1):
        super().__init__()
        self.conv1 = supermaskconv3x3(in_channels, channels, stride)
        self.bn1 = NonAffineBatchNorm(channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = supermaskconv3x3(channels, channels)
        self.bn2 = NonAffineBatchNorm(channels)
        # 入力と出力のチャンネル数が異なる場合（strideが1より大きい場合）、ダウンサンプリング
        if in_channels != channels * self.expansion or stride > 1:
            self.shortcut = nn.Sequential(
                supermaskconv1x1(in_channels, channels * self.expansion, stride),
                NonAffineBatchNorm(channels * self.expansion)
            )
        else:
            self.shortcut = nn.Sequential()

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        # 残差写像と恒等写像の和を計算
        out += self.shortcut(x)
        out = self.relu(out)
        return out

In [23]:
# サブネットワーク獲得用のResNetを定義
class SupermaskResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super().__init__()
        self.in_channels = 64
        self.conv1 = SupermaskConv(3, self.in_channels, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = NonAffineBatchNorm(self.in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        # Residual Blocks(in_channels=64)
        self.layer1 = self._make_layer(block, 64, layers[0], stride=1)
        # Residual Blocks(in_channels=128)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        # Residual Blocks(in_channels=256)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        # Residual Blocks(in_channels=512)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        # 重みの初期化
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                # 初期化手法を設定
                m.set_init(init)
                # 刈り込み率を設定
                m.set_prune_rate(prune_rate)
                # 重みを初期化
                m._init_conv()

    # Residual Blocksを作成する関数を定義
    def _make_layer(self, block, channels, blocks, stride):
        layers = []
        # 最初のResidual Block（stride=stride）
        layers.append(block(self.in_channels, channels, stride))
        # 残りのResidual Block（stride=1）
        self.in_channels = channels * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

In [24]:
# モデルを呼び出す関数を定義
def supermaskresnet34_100():
    return SupermaskResNet(SupermaskBasicBlock, [3, 4, 6, 3], num_classes=100)

## 学習と評価（100 epochs, init=kaiming_uniform）

### prune_rate=0.999

In [29]:
# 刈り込み率を設定
prune_rate = 0.999
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR100_100epochs_pr0.999_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 1.364 %, Loss: 4.5896
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 2.568 %, Loss: 4.3765
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 3.438 %, Loss: 4.3820
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 4.536 %, Loss: 4.2638
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 4.816 %, Loss: 4.1583
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 4.954 %, Loss: 4.1583
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 5.352 %, Loss: 4.1374
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 5.63 %, Loss: 4.1230
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 6.256 %, Loss: 4.0982
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 6.47 %, Loss: 4.0293
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 6.7 %, Loss: 4.

In [30]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [31]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 10.76 %


In [32]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [25]:
# 刈り込み率を設定
prune_rate = 0.995
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR100_100epochs_pr0.995_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 2.978 %, Loss: 4.2727
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 7.154 %, Loss: 3.9263
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 8.566 %, Loss: 3.8573
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 9.544 %, Loss: 3.8577
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 10.476 %, Loss: 3.7734
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 11.43 %, Loss: 3.6898
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 11.848 %, Loss: 3.6850
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 12.642 %, Loss: 3.7468
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 13.556 %, Loss: 3.5634
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 14.076 %, Loss: 3.4505
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 14.71 %,

In [26]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [27]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 21.72 %


In [28]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [ ]:
# 刈り込み率を設定
prune_rate = 0.99
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR100_100epochs_pr0.99_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 4.318 %, Loss: 4.0909
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 9.304 %, Loss: 3.8143
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 10.798 %, Loss: 3.6695
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 11.97 %, Loss: 3.7300
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 13.25 %, Loss: 3.6409
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 14.148 %, Loss: 3.4991
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 15.368 %, Loss: 3.4150
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 16.248 %, Loss: 3.5107
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 17.206 %, Loss: 3.3126
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 18.304 %, Loss: 3.2585
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 19.024 

In [ ]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [ ]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 23.36 %


In [ ]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [ ]:
# 刈り込み率を設定
prune_rate = 0.95
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR100_100epochs_pr0.95_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 6.77 %, Loss: 3.7805
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 12.628 %, Loss: 3.5989
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 15.022 %, Loss: 3.5090
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 17.344 %, Loss: 3.4858
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 18.914 %, Loss: 3.1834
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 20.73 %, Loss: 3.3059
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 22.36 %, Loss: 3.1804
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 23.362 %, Loss: 3.1049
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 24.808 %, Loss: 2.9598
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 25.766 %, Loss: 2.8900
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 26.926 

In [ ]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [ ]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 37.01 %


In [ ]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [ ]:
# 刈り込み率を設定
prune_rate = 0.9
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR100_100epochs_pr0.9_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 6.838 %, Loss: 3.7919
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 13.24 %, Loss: 3.5562
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 15.656 %, Loss: 3.4965
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 18.746 %, Loss: 3.3945
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 20.67 %, Loss: 3.2021
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 22.75 %, Loss: 2.9480
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 23.934 %, Loss: 2.8979
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 25.846 %, Loss: 2.9297
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 27.078 %, Loss: 2.8781
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 28.238 %, Loss: 2.6883
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 29.104 

In [ ]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [ ]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 42.81 %


In [ ]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [ ]:
# 刈り込み率を設定
prune_rate = 0.7
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR100_100epochs_pr0.7_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 5.936 %, Loss: 3.8046
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 12.872 %, Loss: 3.6545
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 16.022 %, Loss: 3.3499
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 18.414 %, Loss: 3.2270
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 20.936 %, Loss: 3.2170
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 22.776 %, Loss: 3.1438
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 24.576 %, Loss: 2.9082
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 26.392 %, Loss: 2.9481
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 27.788 %, Loss: 2.8020
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 29.182 %, Loss: 2.8366
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 30.6

In [ ]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [ ]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 48.29 %


In [ ]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [ ]:
# 刈り込み率を設定
prune_rate = 0.5
# 初期化手法を設定
init = 'kaiming_uniform'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR100_100epochs_pr0.5_uniform_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 6.58 %, Loss: 3.8850
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 12.66 %, Loss: 3.5980
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 15.266 %, Loss: 3.4215
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 18.392 %, Loss: 3.3472
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 21.074 %, Loss: 3.1981
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 23.038 %, Loss: 2.9377
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 25.258 %, Loss: 3.0008
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 27.312 %, Loss: 2.9537
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 29.004 %, Loss: 2.6725
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 29.968 %, Loss: 2.6745
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 31.24 

In [ ]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [ ]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 48.35 %


In [ ]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

## 学習と評価（100 epochs, init=kaiming_normal）

### prune_rate=0.999

In [37]:
# 刈り込み率を設定
prune_rate = 0.999
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR100_100epochs_pr0.999_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 1.28 %, Loss: 4.5903
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 3.03 %, Loss: 4.3321
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 3.616 %, Loss: 4.2321
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 4.514 %, Loss: 4.1817
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 4.818 %, Loss: 4.1588
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 5.634 %, Loss: 4.1619
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 5.984 %, Loss: 4.1504
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 6.18 %, Loss: 4.0519
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 6.424 %, Loss: 4.0963
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 6.726 %, Loss: 4.0445
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 6.928 %, Loss: 4

In [38]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [39]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 6.41 %


In [40]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [33]:
# 刈り込み率を設定
prune_rate = 0.995
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR100_100epochs_pr0.995_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 2.828 %, Loss: 4.1418
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 7.122 %, Loss: 4.0016
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 8.37 %, Loss: 3.8934
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 8.934 %, Loss: 3.8902
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 9.596 %, Loss: 3.9746
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 10.09 %, Loss: 3.7232
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 11.066 %, Loss: 3.7735
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 11.792 %, Loss: 3.7407
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 12.482 %, Loss: 3.7778
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 13.128 %, Loss: 3.5686
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 13.11 %, L

In [34]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [35]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 9.57 %


In [36]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [ ]:
# 刈り込み率を設定
prune_rate = 0.99
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR100_100epochs_pr0.99_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 3.772 %, Loss: 4.0816
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 8.964 %, Loss: 3.9295
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 10.24 %, Loss: 3.7431
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 11.176 %, Loss: 3.7549
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 12.276 %, Loss: 3.6827
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 13.218 %, Loss: 3.5225
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 14.474 %, Loss: 3.6348
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 15.314 %, Loss: 3.5161
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 15.906 %, Loss: 3.3194
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 17.22 %, Loss: 3.4194
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 18.02 %

In [ ]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [ ]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 23.82 %


In [ ]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [13]:
# 刈り込み率を設定
prune_rate = 0.95
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR100_100epochs_pr0.95_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 6.332 %, Loss: 3.9570
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 12.516 %, Loss: 3.6063
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 14.504 %, Loss: 3.4455
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 16.558 %, Loss: 3.3475
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 18.58 %, Loss: 3.2690
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 20.516 %, Loss: 3.2038
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 21.51 %, Loss: 2.9931
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 22.96 %, Loss: 3.2304
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 24.106 %, Loss: 2.8885
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 24.782 %, Loss: 2.9612
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 26.136 

In [14]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [15]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 36.67 %


In [16]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [17]:
# 刈り込み率を設定
prune_rate = 0.9
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR100_100epochs_pr0.9_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 6.908 %, Loss: 3.8187
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 13.218 %, Loss: 3.6020
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 15.766 %, Loss: 3.4892
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 18.092 %, Loss: 3.3939
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 20.03 %, Loss: 3.1200
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 21.828 %, Loss: 3.2618
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 23.336 %, Loss: 3.1188
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 24.526 %, Loss: 3.1060
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 26.024 %, Loss: 2.8903
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 26.992 %, Loss: 2.8582
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 28.48

In [18]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [19]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 41.16 %


In [20]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [21]:
# 刈り込み率を設定
prune_rate = 0.7
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR100_100epochs_pr0.7_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 5.892 %, Loss: 3.8946
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 12.356 %, Loss: 3.6147
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 15.406 %, Loss: 3.4677
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 18.512 %, Loss: 3.4718
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 20.552 %, Loss: 3.1652
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 22.894 %, Loss: 2.9362
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 24.636 %, Loss: 2.8585
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 26.71 %, Loss: 2.9069
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 28.118 %, Loss: 2.8214
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 29.338 %, Loss: 2.6819
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 30.68

In [22]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [23]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 48.21 %


In [24]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [25]:
# 刈り込み率を設定
prune_rate = 0.5
# 初期化手法を設定
init = 'kaiming_normal'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR100_100epochs_pr0.5_normal_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 5.136 %, Loss: 3.9238
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 11.222 %, Loss: 3.6901
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 14.808 %, Loss: 3.3630
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 17.602 %, Loss: 3.3790
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 20.112 %, Loss: 3.2040
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 22.49 %, Loss: 3.1845
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 24.444 %, Loss: 2.9207
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 26.384 %, Loss: 3.0019
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 27.7 %, Loss: 2.8171
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 28.802 %, Loss: 2.8310
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 30.36 %

In [26]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [27]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 48.23 %


In [28]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

## 学習と評価（100 epochs, init=signed_constant）

### prune_rate=0.999

In [45]:
# 刈り込み率を設定
prune_rate = 0.999
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR100_100epochs_pr0.999_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 1.274 %, Loss: 4.6005
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 3.04 %, Loss: 4.4106
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 3.96 %, Loss: 4.2708
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 4.502 %, Loss: 4.1904
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 4.848 %, Loss: 4.1974
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 5.532 %, Loss: 4.1867
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 6.396 %, Loss: 4.1017
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 6.646 %, Loss: 4.0348
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 6.106 %, Loss: 3.9371
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 7.064 %, Loss: 4.1534
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 7.29 %, Loss: 4

In [46]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [47]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 7.5 %


In [48]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.995

In [41]:
# 刈り込み率を設定
prune_rate = 0.995
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR100_100epochs_pr0.995_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 3.416 %, Loss: 4.1875
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 8.11 %, Loss: 3.8964
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 9.276 %, Loss: 3.9147
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 10.356 %, Loss: 3.8781
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 11.69 %, Loss: 3.7582
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 12.53 %, Loss: 3.6185
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 13.572 %, Loss: 3.6271
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 14.152 %, Loss: 3.5351
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 15.172 %, Loss: 3.6533
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 15.608 %, Loss: 3.5270
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 16.492 %,

In [42]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [43]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 26.27 %


In [44]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.99

In [29]:
# 刈り込み率を設定
prune_rate = 0.99
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR100_100epochs_pr0.99_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 4.944 %, Loss: 4.0288
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 10.572 %, Loss: 3.7177
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 11.8 %, Loss: 3.7044
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 12.956 %, Loss: 3.7063
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 14.596 %, Loss: 3.6088
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 15.99 %, Loss: 3.3847
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 16.942 %, Loss: 3.5264
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 18.344 %, Loss: 3.3403
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 19.37 %, Loss: 3.3929
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 19.842 %, Loss: 3.2409
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 20.97 %,

In [30]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [31]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 30.87 %


In [32]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.95

In [33]:
# 刈り込み率を設定
prune_rate = 0.95
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR100_100epochs_pr0.95_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 7.096 %, Loss: 3.8738
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 13.572 %, Loss: 3.6398
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 15.84 %, Loss: 3.4169
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 18.27 %, Loss: 3.2708
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 19.934 %, Loss: 3.1973
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 21.892 %, Loss: 3.1174
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 23.432 %, Loss: 3.1956
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 24.71 %, Loss: 3.0100
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 26.418 %, Loss: 3.0160
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 27.416 %, Loss: 2.8416
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 28.67 %

In [34]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [35]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 43.08 %


In [36]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.9

In [37]:
# 刈り込み率を設定
prune_rate = 0.9
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR100_100epochs_pr0.9_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 7.322 %, Loss: 3.6560
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 13.996 %, Loss: 3.3987
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 16.824 %, Loss: 3.4257
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 19.6 %, Loss: 3.3054
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 21.322 %, Loss: 3.0828
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 23.464 %, Loss: 3.1014
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 24.854 %, Loss: 2.9416
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 26.772 %, Loss: 2.6547
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 28.294 %, Loss: 2.6328
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 29.944 %, Loss: 2.8905
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 30.704

In [38]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [39]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 46.77 %


In [40]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.7

In [13]:
# 刈り込み率を設定
prune_rate = 0.7
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR100_100epochs_pr0.7_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 6.582 %, Loss: 3.8314
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 13.45 %, Loss: 3.5843
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 16.662 %, Loss: 3.3804
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 19.46 %, Loss: 3.2293
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 22.304 %, Loss: 3.1601
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 24.678 %, Loss: 2.9548
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 26.648 %, Loss: 2.7929
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 28.788 %, Loss: 2.7731
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 29.93 %, Loss: 2.9130
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 31.39 %, Loss: 2.8047
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 32.83 %,

In [14]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [15]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 51.15 %


In [16]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()

### prune_rate=0.5

In [17]:
# 刈り込み率を設定
prune_rate = 0.5
# 初期化手法を設定
init = 'signed_constant'

# デバイスを設定
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# モデルを定義
model = supermaskresnet34_100().to(device)
# 学習前のモデルの重みを保存
model_init = copy.deepcopy(model)

# 学習率を設定
learning_rate = 0.001
# 全体のepoch数を設定
num_epochs = 100
# warm-upするepoch数を設定
warmup_epochs = 10

# 損失関数を定義
criterion = nn.CrossEntropyLoss()
# オプティマイザーを設定
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# スケジューラーを設定
scheduler = CosineLRScheduler(optimizer, t_initial=num_epochs, lr_min=1e-4, warmup_t=warmup_epochs, warmup_lr_init=1e-4, warmup_prefix=True)

# モデルの学習
for epoch in range(num_epochs):
    total = 0
    correct = 0
    for i, (images, labels) in enumerate(train_loader_cifar100):
        images = images.to(device)
        labels = labels.to(device)
        # 順伝播
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 逆伝播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        _, predicted = torch.max(outputs.data, 1)
        # 全データ数
        total += labels.size(0)
        # 正解数
        correct += (predicted == labels).sum().item()
    # 正解率精度を計算
    train_acc = 100 * correct / total
    # 正解率精度と損失を確認
    print("Epoch [{}/{}], Train Accuracy: {} %, Loss: {:.4f}".format(epoch+1, num_epochs, 100 * correct / total, loss.item()))
    # 1エポック終了後にスケジューラーを更新
    scheduler.step(epoch)
    # 学習率の確認
    print("Epoch [{}/{}], Learning Rate: {}".format(epoch+1, num_epochs, optimizer.param_groups[0]['lr']))

# モデルを保存
torch.save(model, '/content/drive/MyDrive/SupermaskResNet34_CIFAR100_100epochs_pr0.5_constant_CLRS_restest.pth')

Epoch [1/100], Train Accuracy: 6.08 %, Loss: 3.9397
Epoch [1/100], Learning Rate: 0.0001
Epoch [2/100], Train Accuracy: 12.396 %, Loss: 3.6436
Epoch [2/100], Learning Rate: 0.00019
Epoch [3/100], Train Accuracy: 15.54 %, Loss: 3.5387
Epoch [3/100], Learning Rate: 0.00028
Epoch [4/100], Train Accuracy: 19.126 %, Loss: 3.4039
Epoch [4/100], Learning Rate: 0.00036999999999999994
Epoch [5/100], Train Accuracy: 21.818 %, Loss: 3.1603
Epoch [5/100], Learning Rate: 0.00045999999999999996
Epoch [6/100], Train Accuracy: 24.204 %, Loss: 2.9829
Epoch [6/100], Learning Rate: 0.00055
Epoch [7/100], Train Accuracy: 26.156 %, Loss: 2.6751
Epoch [7/100], Learning Rate: 0.0006399999999999999
Epoch [8/100], Train Accuracy: 27.972 %, Loss: 2.7744
Epoch [8/100], Learning Rate: 0.00073
Epoch [9/100], Train Accuracy: 29.786 %, Loss: 2.6732
Epoch [9/100], Learning Rate: 0.00082
Epoch [10/100], Train Accuracy: 31.646 %, Loss: 2.6330
Epoch [10/100], Learning Rate: 0.00091
Epoch [11/100], Train Accuracy: 32.626

In [18]:
# 学習前後のモデルの重みを比較する関数を作成
def check_weight_change(model, model_init):
    weights_changed = (model.conv1.state_dict()['weight'] != model_init.conv1.state_dict()['weight']).any()
    if weights_changed:
        return print('モデルの重みが変化しています')
    else:
        return print('モデルの重みは変化していません')
# 学習前後でモデルの重みが変化していないかを確認
check_weight_change(model, model_init)

モデルの重みは変化していません


In [19]:
# モデルの評価
model.eval()
with torch.no_grad():
    total = 0
    correct = 0
    for images, labels in test_loader_cifar100:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    # 正解率精度の確認
    print('Test Accuracy: {} %'.format(100 * correct / total))

Test Accuracy: 50.83 %


In [20]:
# GPUメモリの解放
del model, model_init
torch.cuda.empty_cache()